# Data Preparation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px

In [2]:
df = pd.read_csv('data/hotel_booking.csv')

In [3]:
# drop company, agent, name, email, phone-number, and credit_card
df.drop(['company', 'agent', 'name', 'email', 'phone-number', 'credit_card'], axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [5]:
numerical = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests']
categorical = ['hotel', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type']

In [6]:
df.isnull().sum()

hotel                               0
is_canceled                         0
lead_time                           0
arrival_date_year                   0
arrival_date_month                  0
arrival_date_week_number            0
arrival_date_day_of_month           0
stays_in_weekend_nights             0
stays_in_week_nights                0
adults                              0
children                            4
babies                              0
meal                                0
country                           488
market_segment                      0
distribution_channel                0
is_repeated_guest                   0
previous_cancellations              0
previous_bookings_not_canceled      0
reserved_room_type                  0
assigned_room_type                  0
booking_changes                     0
deposit_type                        0
days_in_waiting_list                0
customer_type                       0
adr                                 0
required_car

In [7]:
df.shape

(119390, 30)

## Remove Missing Value

In [8]:
undefined_counts = (df == "Undefined").sum()
print(undefined_counts[undefined_counts > 0])

meal                    1169
market_segment             2
distribution_channel       5
dtype: int64


In [9]:
df['meal'] = df['meal'].replace("Undefined", pd.NA)
df['distribution_channel'] = df['distribution_channel'].replace("Undefined", pd.NA)

In [10]:
df.dropna(inplace=True)

## Miscellaneous

In [11]:
# Combine countries with small samples
threshold = 500
df['country'] = df['country'].apply(lambda x: x if df['country'].value_counts()[x] >= threshold else 'Other')

In [17]:
df['country'].value_counts()

country
PRT      47886
GBR      12109
FRA      10324
ESP       8271
DEU       7286
Other     6879
ITA       3763
IRL       3374
BEL       2341
BRA       2222
NLD       2104
USA       2096
CHE       1728
CN        1270
AUT       1263
CHN        999
SWE        990
POL        919
ISR        669
RUS        632
NOR        607
Name: count, dtype: int64

In [13]:
# reservation status and date would be known with cancellation, which will bias the model
df.drop(['reservation_status', 'reservation_status_date'], axis=1, inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117732 entries, 0 to 119389
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           117732 non-null  object 
 1   is_canceled                     117732 non-null  int64  
 2   lead_time                       117732 non-null  int64  
 3   arrival_date_year               117732 non-null  int64  
 4   arrival_date_month              117732 non-null  object 
 5   arrival_date_week_number        117732 non-null  int64  
 6   arrival_date_day_of_month       117732 non-null  int64  
 7   stays_in_weekend_nights         117732 non-null  int64  
 8   stays_in_week_nights            117732 non-null  int64  
 9   adults                          117732 non-null  int64  
 10  children                        117732 non-null  float64
 11  babies                          117732 non-null  int64  
 12  meal                 

## Train-Test Split with SMOTE

To prevent the model from becoming biased toward the majority class of the training set for each fold, we will apply <a href='https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html'>Synthetic Minority Over-sampling Technique (SMOTE)</a>, which creates synthetic data points for the minority class with the same features' distribution of the original dataset. This helps the model to learn from a more balanced dataset. SMOTE will be applied to only training set to prevent data leakage.

In [15]:
# Export cleaned dataset
df.to_csv('./data/hotel_cleaned.csv', index=False)

In [20]:
for col in df.describe(include='object').columns:
    print(col)
    print(df[col].unique())
    print('-'*50)

hotel
['Resort Hotel' 'City Hotel']
--------------------------------------------------
arrival_date_month
['July' 'August' 'September' 'October' 'November' 'December' 'January'
 'February' 'March' 'April' 'May' 'June']
--------------------------------------------------
meal
['BB' 'FB' 'HB' 'SC']
--------------------------------------------------
country
['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'Other' 'NOR' 'POL' 'DEU' 'BEL' 'CHE'
 'CN' 'ITA' 'NLD' 'RUS' 'SWE' 'BRA' 'CHN' 'AUT' 'ISR']
--------------------------------------------------
market_segment
['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary' 'Groups'
 'Aviation']
--------------------------------------------------
distribution_channel
['Direct' 'Corporate' 'TA/TO' 'GDS']
--------------------------------------------------
reserved_room_type
['C' 'A' 'D' 'E' 'G' 'F' 'H' 'L' 'B' 'P']
--------------------------------------------------
assigned_room_type
['C' 'A' 'D' 'E' 'G' 'F' 'I' 'B' 'H' 'L' 'K' 'P']
-------------------

In [19]:
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split

X = df.drop(['is_canceled'], axis=1)
y = df['is_canceled']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=14)

# Get categorical feature indices for SMOTENC
cat_indices = [X_train.columns.get_loc(col) for col in categorical if col in X_train.columns]

# Apply SMOTE before encoding
smote = SMOTENC(random_state=42, categorical_features=cat_indices)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Apply One-Hot Encoding
one_hot_cols = ['country', 'hotel', 'arrival_date_month', 'meal', 'market_segment', 
                'distribution_channel', 'reserved_room_type', 'assigned_room_type', 
                'deposit_type', 'customer_type']
X_resampled = pd.get_dummies(X_resampled, columns=one_hot_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=one_hot_cols, drop_first=True)

# Ensure train and test sets have the same features
X_resampled, X_test = X_resampled.align(X_test, join='left', axis=1, fill_value=0)

# Export train-test split dataset
train_data = pd.concat([X_resampled, y_resampled], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv(f'./data/train-test/train.csv', index=False)
test_data.to_csv(f'./data/train-test/test.csv', index=False)